# Download, Categorize, and Filter Papers
Refer https://github.com/NisansaDdS/Some-Languages-are-More-Equal-than-Others/blob/main/Generate%20Diagrams.ipynb for downloading anthology.bib and generating ACL_Abstracts.txt and ACL_URLs.txt

In [9]:
import inflect
import re
import os
from urllib.request import urlretrieve
from tqdm import tqdm

In [10]:
aclBibPath = "anthology.bib"

In [11]:
with open(aclBibPath, "r", encoding="utf-8") as fp:
    bib_txt = fp.read()

In [12]:
bibs = bib_txt.split("}\n@")

In [13]:
def replace_non_letters_with_spaces(input_string):
    # Replace non-letter characters with spaces
    result_string = re.sub(r'[^a-zA-ZÀ-ÿ ]', ' ', input_string)
    return result_string

In [27]:
with open("ACL_Abstracts.txt", "r") as fp:
    abstracts = fp.read().split("\n")[:-1]

In [48]:
lang_classes = {}
for i in range(6):
    with open(f"LangClasses/{i}.txt", "r", encoding="utf-8") as fp:
        lang_classes[i] = set(fp.read().strip().split("\n"))

In [ ]:
languages_to_ignore ="Apache,Laura,Fang,Mono,Ma,Maria,Sam,Bench,Zhuang,Male,Nara,So,Hu,Kim,Label,The,To,Yong,The,To,Adele,Are,Foma,Kaur,Bau,Kato,Dek,Naman,Dom,As,The,To,As,Dan,E,The,To,U,Even,En,Chung,Dong,Shi,Tai,Thompson,Gao,Ir,Pan,Ali,Rao,Han,Doe,Titan,Ha,Sa,Tu,Lau,Siri,Wan,She,Dai,Ding,Kang,Ge,Koch,Che,Mann,Zou,Pei,Yao,Lou,Sydney,Ju,Sha,Day,Miwa,Bai,Ko,Ga,Pal,Pe,Gun,Hung,Con,Cun,Serrano,Sui,Bu,Mehri,Od,Haji,Gal,Gey,Lui,Ho,Furu,Ak,Kao,Aro,Gen,Moro,Notre,Ido,Ron,Were,Bai,Sahu,Dem,Melo,Rama,Hunde,Dii,Yala,Sauri".split(',')
languages_to_ignore=languages_to_ignore+"Uni,One,Yi,Na,Bit,Pa".split(',') + ["are", "as", "e", "en", "even", "one", "so", "to", "apache", "au", "u", "bit", "she", "siri", "day", "gun", "label", "notre"]
languages_to_ignore = set(languages_to_ignore)

In [60]:
url_cls = {}
url_langs = {}
for line in tqdm(abstracts):
    title, url, abstract = line.split("\t")
    for cls, langs in lang_classes.items():
        proc_abs = set(replace_non_letters_with_spaces(abstract.replace("{", "").replace("}", "")).split(" "))
        for lang in langs:
            if lang in proc_abs and lang not in languages_to_ignore:
                url_cls[url] = cls
                url_langs[url] = lang
                break
        else:
            continue
        break

  0%|          | 0/86948 [00:00<?, ?it/s]

100%|██████████| 86948/86948 [00:09<00:00, 9285.36it/s] 


In [61]:
p = inflect.engine()

conf=["first",'second','third','fourth','fifth','sixth',"seventh","eighth","ninth","tenth","eleventh","twelfth","thirteenth","fourteenth","fifteenth","sixteenth","seventeenth","eighteenth","nineteenth","twentieth"]

capital=[c.title() for c in conf]

conf=conf+capital

mainVenues=[]

for i in range(70):
    conf.append(p.ordinal(i+1))

conf=[" "+c+" " for c in conf]

f=open(aclBibPath, "r", encoding="utf-8")

def simplyfyVenue(x):
  for c in conf:
    if c in x:
      x=x.split(c)[1]
      break
  x=x.split("(")[0].strip()
  for i in range(1900,2021):
    if str(i) in x:
      x=x.split(str(i))[1].strip()
  return(x)

mainVenues=["Main Conference",
            "Annual Meeting of the Association for Computational Linguistics",
            "North American Chapter of the Association for Computational Linguistics",
            "{E}uropean Chapter of the Association for Computational Linguistics",
            "Empirical Methods in Natural Language Processing",
            "International Conference on Computational Linguistics",
            "Conference on Computational Natural Language Learning",
            "International Workshop on Semantic Evaluation",
            "Conference of the Asia-Pacific Chapter of the Association for Computational Linguistics",
            "Conference on Computational Natural Language Learning"]

def mapVanue(x,isJournal):
  if ("Language Resources and Evaluation" in x) or ("LREC" in x):
    return("LREC")
  elif any([v in x for v in mainVenues]):
    mainVenues.append(x)
    return("Main")
  elif isJournal and (("Transactions of the Association for Computational Linguistics" in x) or ("Computational Linguistics" in x)):
    return("Main")
  else:
    return("Other")

def defang(x):
  s=x[x.index("\"")+1:x.rindex("\"")]
  if(len(s)==0):
    s=x[x.index("{")+1:x.rindex("}")]
  return(s)

paperDetails={}
categories = {}
counter=0


for x in f:
  if("booktitle" not in x)and("title =" in x or "Title =" in x):
    title=defang(x)
    counter=1
  elif("booktitle" in x):
    place=defang(x)
    place=mapVanue(place,False)
    counter=counter+1
  elif ("journal =" in x):
    place=defang(x)
    place=mapVanue(place,True)
    counter=counter+1
  elif("year =" in x):
    year=int(defang(x))
    counter=counter+1
  elif("url =" in x):
    url=defang(x)
    counter=counter+1
  if(counter==4):
    paperDetails[title]=[place,year,url]
    if year >= 2015 and url in url_cls:
      lang_class = url_cls[url]
      if place not in categories:
        categories[place] = {}
      if lang_class not in categories[place]:
        categories[place][lang_class] = {}
      if year not in categories[place][lang_class]:
        categories[place][lang_class][year] = set()
      categories[place][lang_class][year].add(url)
    counter=0

mainVenues=list(set(mainVenues))

In [54]:
# Cache for later use
import pickle

with open("categories_f.pkl", "wb") as fp:
    pickle.dump(categories, fp)

In [ ]:
import shutil
from tqdm import tqdm
from copy import deepcopy

samples = deepcopy(categories)
for category in samples:
  for lang_class in samples[category]:
    print(category, lang_class)
    for year in tqdm(samples[category][lang_class]):
      save_dir = os.path.join("CategoryFiltered/", category, str(lang_class), str(year))
      for url in samples[category][lang_class][year]:
        url += "" if url.endswith(".pdf") else ".pdf"
        if not os.path.exists(save_dir):
          os.makedirs(save_dir, exist_ok=True)
        pdf_path = os.path.join("PDFs", os.path.basename(url))
        txt_path = pdf_path.replace("PDFs", "TXTs").replace(".pdf", ".txt")
        dst_pdf_path = os.path.join(save_dir, os.path.basename(url))
        dst_txt_path = dst_pdf_path.replace(".pdf", ".txt")
        if os.path.exists(txt_path):
          shutil.copy(pdf_path, dst_pdf_path)
          shutil.copy(txt_path, dst_txt_path)
        else:
          try:
            urlretrieve(url, os.path.join(save_dir, os.path.basename(url)))
          except Exception as e:
            print(f"{category}, {lang_class}, {year}, {url}", e)

In [146]:
from glob import glob

txts = set(map(lambda x: os.path.basename(x), glob("TXTs/*/*.txt")))

seen = set()
not_seen = set()

for category in samples:
  for lang_class in samples[category]:
    for year in samples[category][lang_class]:
      pdf_dir = os.path.join(save_dir, category, str(lang_class), str(year))
      for url in samples[category][lang_class][year]:
        url += "" if url.endswith(".pdf") else ".pdf"
        url = url.replace(".pdf", ".txt")
        if os.path.basename(url) in txts:
            seen.add(os.path.basename(url))
        else:
            not_seen.add(os.path.basename(url))

In [148]:
with open("in_filtered.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(sorted(seen)))
with open("not_in_filtered.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(sorted(not_seen)))

In [9]:
from glob import glob

pdf_papers = glob("PDFs/*/*.pdf")
txt_papers = glob("TXTs/*/*.txt")

In [7]:
len(pdf_papers)

26001

In [10]:
len(txt_papers)

17931

# Sample filtered papers

In [76]:
import os
from glob import glob
from shutil import copy
import random

random.seed(0)

In [77]:
with open("LangClasses/5.txt", "r", encoding="utf-8") as fp:
    class_5_langs = fp.read().strip().split("\n")
seen_dir = "LangClasses/5/" # If previously sampled papers exist
out_dir = "LangClasses/5-new/" # For new sampled papers

In [78]:
pdf_paths = list(map(os.path.basename, glob("CategoryFiltered/*/5/202*/*.pdf")))
seen_pdfs = list(map(os.path.basename, glob(os.path.join(seen_dir, "*.pdf"))))

In [73]:
for lang in class_5_langs:
    txt_paths = glob(f"TXTs/{lang}/*.txt")
    sample_paths = random.sample(txt_paths, k=20)
    for txt_path in sample_paths:
        pdf_path = txt_path.replace("TXTs/", "PDFs/").replace(".txt", ".pdf")
        dst_txt_path = os.path.join(out_dir, lang + "_" + os.path.basename(txt_path))
        dst_pdf_path = os.path.join(out_dir, lang + "_" + os.path.basename(pdf_path))
        if os.path.basename(pdf_path) in pdf_paths:
            copy(txt_path, dst_txt_path)
            copy(pdf_path, dst_pdf_path)

In [75]:
for lang in class_5_langs:
    txt_paths = glob(f"TXTs/{lang}/*.txt")
    tot = 0
    indices = list(range(len(txt_paths)))
    seen = set()
    while tot < 20:
        sample_idx = random.sample(indices, k = 1)[0]
        if sample_idx in seen:
            continue
        seen.add(sample_idx)
        txt_path = txt_paths[sample_idx]
        pdf_path = txt_path.replace("TXTs/", "PDFs/").replace(".txt", ".pdf")
        pdf_name = os.path.basename(pdf_path)
        dst_txt_path = os.path.join(out_dir, lang + "_" + os.path.basename(txt_path))
        dst_pdf_path = os.path.join(out_dir, lang + "_" + pdf_name)
        if pdf_name in pdf_paths:
            if os.path.basename(dst_pdf_path) in seen_pdfs:
                continue
            copy(txt_path, dst_txt_path)
            copy(pdf_path, dst_pdf_path)
            tot += 1

# Sample for manual verification

In [9]:
import os
from glob import glob
from shutil import copy
import random

random.seed(0)

In [16]:
pdf_paths = glob("PDFs/*/*.pdf")
sample_paths = random.sample(pdf_paths, k=100)
out_dir = "ManualVerifySample/"

In [17]:
for path in sample_paths:
    dst_pdf_path = os.path.join(out_dir, os.path.basename(path))
    copy(path, dst_pdf_path)
    
    txt_path = path.replace("PDFs", "TXTs").replace(".pdf", ".txt")
    if os.path.exists(txt_path):
        dst_txt_path = os.path.join(out_dir, os.path.basename(txt_path))
        copy(txt_path, dst_txt_path)